# Практическое задание к уроку 13. Модель BERT и GPT


1. Взять датасет

https://huggingface.co/datasets/merionum/ru_paraphraser

решить задачу парафраза

2. **(дополнительно необязательная задача)**

на выбор взять

https://huggingface.co/datasets/sberquad

https://huggingface.co/datasets/blinoff/medical_qa_ru_data

натренировать любую модель для вопросно ответной системы

как альтернатива можно взять любой NER датасет из 

https://github.com/natasha/corus#reference и обучить NER

**1. Взять датасет https://huggingface.co/datasets/merionum/ru_paraphraser, решить задачу парафраза**

In [1]:
!pip install transformers
!pip install datasets
!pip install transformers[sentencepiece]

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Загрузка датасета

from datasets import load_dataset
dataset = load_dataset("merionum/ru_paraphraser")
dataset

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 28.5 MB/s 
     |████████████████████████████████| 101 kB 14.4 MB/s 
     |████████████████████████████████| 6.6 MB 59.5 MB/s 
     |████████████████████████████████| 596 kB 65.3 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 365 kB 31.4 MB/s 
     |████████████████████████████████| 141 kB 70.6 MB/s 
     |████████████████████████████████| 212 kB 70.8 MB/s 
     |████████████████████████████████| 115 kB 74.9 MB/s 
     |████████████████████████████████| 127 kB 60.1 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

0 tables [00:00, ? tables/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/merionum___json/merionum--ru_paraphraser-aad01005423eb70c/0.0.0/a3e658c4731e59120d44081ac10bf85dc7e1388126b92338344ce9661907f253. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'id_1', 'id_2', 'text_1', 'text_2', 'class'],
        num_rows: 7227
    })
    test: Dataset({
        features: ['id', 'id_1', 'id_2', 'text_1', 'text_2', 'class'],
        num_rows: 1924
    })
})

In [2]:
dataset['train'][1]

{'id': '2',
 'id_1': '202',
 'id_2': '8158',
 'text_1': 'Право полицейских на проникновение в жилище решили ограничить.',
 'text_2': 'Правила внесудебного проникновения полицейских в жилище уточнят.',
 'class': '0'}

In [3]:
# Загрузка предобученных моделей и пайплайна
tokenizer = AutoTokenizer.from_pretrained("cointegrated/rut5-base-paraphraser")
model = AutoModelForSeq2SeqLM.from_pretrained("cointegrated/rut5-base-paraphraser")

/usr/local/lib/python3.7/dist-packages/transformers/convert_slow_tokenizer.py:435: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  "The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option"


In [4]:
text = 'Вучич заявил об аде на Земле в случае отказа Запада от переговоров с Путиным'

In [5]:
x = tokenizer(text, return_tensors='pt', padding=True).to(model.device)
x

{'input_ids': tensor([[  635, 15376,  1248,   374, 19126,  1091,  1981,   324,   310, 11071,
          1934,   315,  2536,   324,  8786,   308,  1145,  8988,   552, 26967,
         22110,   388,   259, 10086,  1689,     1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1]])}

In [6]:
for inx2tok in x['input_ids'][0]:
  print(tokenizer.decode(inx2tok))

В
учи
ч
за
явил
об
ад
е
на
Зем
ле
в
случа
е
отказ
а
За
пада
от
перего
воров
с

Путин
ым
</s>


In [7]:
# Функция парафраза

def paraphrase(text, beams=15, grams=2, do_sample=False):
    x = tokenizer(text, return_tensors='pt', padding=True).to(model.device)
    print(x, x.input_ids.shape[1])
    max_size = int(x.input_ids.shape[1] * 1.5 + 10)
    out = model.generate(**x, encoder_no_repeat_ngram_size=grams, num_beams=beams, max_length=max_size, do_sample=do_sample)
    return tokenizer.decode(out[0], skip_special_tokens=True)

In [8]:
import random

In [9]:
# Примеры работы парафраза grams = 6

for i in range(5):
    k = random.randint(1,100)
    print('Исходный текст: ', dataset["train"][k]['text_1'])
    print('Парафраз: ', paraphrase(dataset["train"][k]['text_1'], grams=6))
    print('------------------------------------------------------------')

Исходный текст:  В Египте приговорили к смерти участников погрома на стадионе в Порт-Саиде.
{'input_ids': tensor([[  635, 11865, 29273,   805, 14466,   279,   778,  8750,   279,  2904,
          4277, 20164, 23157,   310, 13418,  9183,   315, 17917,   264, 12436,
         16815,   260,     1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])} 23
Парафраз:  В Египте приговорены к смерти участников погрома на стадионе Порт-Саид.
------------------------------------------------------------
Исходный текст:  Москвичи смогут забронировать в Интернете место на кладбище.
{'input_ids': tensor([[ 9131, 16650,  4247, 11694,   374, 23407,  5059,   315,  8034,   324,
          4200,   310,   259,  4903,  1066,  9174,   260,     1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])} 18
Парафраз:  Москвичи могут забронировать кладбище в интернете.
------------------------------------------------------------
Исходный

In [10]:
# Примеры работы парафраза 
text = 'Вучич заявил об аде на Земле в случае отказа Запада от переговоров с Путиным'
print('Исходный текст: ', text)
print('Парафраз: ', paraphrase(text, grams=4))

Исходный текст:  Вучич заявил об аде на Земле в случае отказа Запада от переговоров с Путиным
{'input_ids': tensor([[  635, 15376,  1248,   374, 19126,  1091,  1981,   324,   310, 11071,
          1934,   315,  2536,   324,  8786,   308,  1145,  8988,   552, 26967,
         22110,   388,   259, 10086,  1689,     1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1]])} 26
Парафраз:  Вучич рассказал о ситуации на Ближнем Востоке в связи с отказом США от переговоров Путина


2. (дополнительно необязательная задача)
на выбор взять

https://huggingface.co/datasets/sberquad

https://huggingface.co/datasets/blinoff/medical_qa_ru_data

**натренировать любую модель для вопросно ответной системы**

как альтернатива можно взять любой NER датасет из

https://github.com/natasha/corus#reference и обучить NER

In [11]:
# Загрузка датасета

from datasets import load_dataset

raw_qa_dataset = load_dataset("sberquad")
# raw_qa_dataset = load_dataset("blinoff/medical_qa_ru_data")

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/45328 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5036 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/23936 [00:00<?, ? examples/s]

Dataset sberquad downloaded and prepared to /root/.cache/huggingface/datasets/sberquad/sberquad/1.0.0/62115d937acf2634cfacbfee10c13a7ee39df3ce345bb45af7088676f9811e77. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [12]:
raw_qa_dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 45328
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 5036
    })
    test: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 23936
    })
})

In [13]:
raw_qa_dataset["train"][0]

{'id': 62310,
 'title': 'SberChallenge',
 'context': 'В протерозойских отложениях органические остатки встречаются намного чаще, чем в архейских. Они представлены известковыми выделениями сине-зелёных водорослей, ходами червей, остатками кишечнополостных. Кроме известковых водорослей, к числу древнейших растительных остатков относятся скопления графито-углистого вещества, образовавшегося в результате разложения Corycium enigmaticum. В кремнистых сланцах железорудной формации Канады найдены нитевидные водоросли, грибные нити и формы, близкие современным кокколитофоридам. В железистых кварцитах Северной Америки и Сибири обнаружены железистые продукты жизнедеятельности бактерий.',
 'question': 'чем представлены органические остатки?',
 'answers': {'text': ['известковыми выделениями сине-зелёных водорослей'],
  'answer_start': [109]}}

In [14]:
# Загрузка предобученного токенайзера

from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("AndrewChar/model-QA-5-epoch-RU")

In [15]:
# Функция препроцессинга тренировочной выборки

max_length = 384
stride = 128


def preprocess_training_examples(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=max_length,
        truncation="only_second",
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    sample_map = inputs.pop("overflow_to_sample_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        sample_idx = sample_map[i]
        answer = answers[sample_idx]
        start_char = answer["answer_start"][0]
        end_char = answer["answer_start"][0] + len(answer["text"][0])
        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label is (0, 0)
        if offset[context_start][0] > start_char or offset[context_end][1] < end_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

In [16]:
# Препроцессинг тренировочной выборки

train_dataset = raw_qa_dataset["train"].map(
    preprocess_training_examples,
    batched=True,
    remove_columns=raw_qa_dataset["train"].column_names,
)

  0%|          | 0/46 [00:00<?, ?ba/s]

In [17]:
# Функция препроцессинга валидационной выборки

def preprocess_validation_examples(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=max_length,
        truncation="only_second",
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    sample_map = inputs.pop("overflow_to_sample_mapping")
    example_ids = []

    for i in range(len(inputs["input_ids"])):
        sample_idx = sample_map[i]
        example_ids.append(examples["id"][sample_idx])

        sequence_ids = inputs.sequence_ids(i)
        offset = inputs["offset_mapping"][i]
        inputs["offset_mapping"][i] = [
            o if sequence_ids[k] == 1 else None for k, o in enumerate(offset)
        ]

    inputs["example_id"] = example_ids
    return inputs

In [18]:
# Препроцессинг валидационной выборки

validation_dataset = raw_qa_dataset["validation"].map(
    preprocess_validation_examples,
    batched=True,
    remove_columns=raw_qa_dataset["validation"].column_names,
)

  0%|          | 0/6 [00:00<?, ?ba/s]

In [19]:
# Для настройки модели возьмем небольшую выборку

small_eval_set = raw_qa_dataset["validation"].select(range(100))

trained_checkpoint = "AndrewChar/model-QA-5-epoch-RU"

tokenizer = AutoTokenizer.from_pretrained(trained_checkpoint)

eval_set = small_eval_set.map(
    preprocess_validation_examples,
    batched=True,
    remove_columns=raw_qa_dataset["validation"].column_names,
)

  0%|          | 0/1 [00:00<?, ?ba/s]

In [20]:
tokenizer = AutoTokenizer.from_pretrained("AndrewChar/model-QA-5-epoch-RU")

In [21]:
# Загрузка предобученной модели

import tensorflow as tf
from transformers import TFAutoModelForQuestionAnswering

eval_set_for_model = eval_set.remove_columns(["example_id", "offset_mapping"])
eval_set_for_model.set_format("numpy")

batch = {k: eval_set_for_model[k] for k in eval_set_for_model.column_names}
trained_model = TFAutoModelForQuestionAnswering.from_pretrained(trained_checkpoint)

outputs = trained_model(**batch)

All model checkpoint layers were used when initializing TFDistilBertForQuestionAnswering.

All the layers of TFDistilBertForQuestionAnswering were initialized from the model checkpoint at AndrewChar/model-QA-5-epoch-RU.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForQuestionAnswering for predictions without further training.


In [22]:
start_logits = outputs.start_logits.numpy()
end_logits = outputs.end_logits.numpy()

In [23]:
import collections

example_to_features = collections.defaultdict(list)
for idx, feature in enumerate(eval_set):
    example_to_features[feature["example_id"]].append(idx)

In [24]:
import numpy as np

n_best = 20
max_answer_length = 30
predicted_answers = []

for example in small_eval_set:
    example_id = example["id"]
    context = example["context"]
    answers = []

    for feature_index in example_to_features[example_id]:
        start_logit = start_logits[feature_index]
        end_logit = end_logits[feature_index]
        offsets = eval_set["offset_mapping"][feature_index]

        start_indexes = np.argsort(start_logit)[-1 : -n_best - 1 : -1].tolist()
        end_indexes = np.argsort(end_logit)[-1 : -n_best - 1 : -1].tolist()
        for start_index in start_indexes:
            for end_index in end_indexes:
                # Skip answers that are not fully in the context
                if offsets[start_index] is None or offsets[end_index] is None:
                    continue
                # Skip answers with a length that is either < 0 or > max_answer_length.
                if (
                    end_index < start_index
                    or end_index - start_index + 1 > max_answer_length
                ):
                    continue

                answers.append(
                    {
                        "text": context[offsets[start_index][0] : offsets[end_index][1]],
                        "logit_score": start_logit[start_index] + end_logit[end_index],
                    }
                )

    best_answer = max(answers, key=lambda x: x["logit_score"])
    predicted_answers.append({"id": example_id, "prediction_text": best_answer["text"]})

In [25]:
# Загрузка метрики

from datasets import load_metric

metric = load_metric("squad")

In [26]:
theoretical_answers = [
    {"id": ex["id"], "answers": ex["answers"]} for ex in small_eval_set
]

In [27]:
# Сравним предсказанный и реальный ответ

print(predicted_answers[0])
print(theoretical_answers[0])

{'id': 60544, 'prediction_text': 'в Древнем Египте'}
{'id': 60544, 'answers': {'text': ['в Древнем Египте'], 'answer_start': [60]}}


In [28]:
# Посмотрим на метрику

metric.compute(predictions=predicted_answers, references=theoretical_answers)

{'exact_match': 61.0, 'f1': 78.85454822954823}

In [29]:
# Функция вычисления метрик

from tqdm.auto import tqdm


def compute_metrics(start_logits, end_logits, features, examples):
    example_to_features = collections.defaultdict(list)
    for idx, feature in enumerate(features):
        example_to_features[feature["example_id"]].append(idx)

    predicted_answers = []
    for example in tqdm(examples):
        example_id = example["id"]
        context = example["context"]
        answers = []

        # Loop through all features associated with that example
        for feature_index in example_to_features[example_id]:
            start_logit = start_logits[feature_index]
            end_logit = end_logits[feature_index]
            offsets = features[feature_index]["offset_mapping"]

            start_indexes = np.argsort(start_logit)[-1 : -n_best - 1 : -1].tolist()
            end_indexes = np.argsort(end_logit)[-1 : -n_best - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Skip answers that are not fully in the context
                    if offsets[start_index] is None or offsets[end_index] is None:
                        continue
                    # Skip answers with a length that is either < 0 or > max_answer_length
                    if (
                        end_index < start_index
                        or end_index - start_index + 1 > max_answer_length
                    ):
                        continue

                    answer = {
                        "text": context[offsets[start_index][0] : offsets[end_index][1]],
                        "logit_score": start_logit[start_index] + end_logit[end_index],
                    }
                    answers.append(answer)

        # Select the answer with the best score
        if len(answers) > 0:
            best_answer = max(answers, key=lambda x: x["logit_score"])
            predicted_answers.append(
                {"id": example_id, "prediction_text": best_answer["text"]}
            )
        else:
            predicted_answers.append({"id": example_id, "prediction_text": ""})

    theoretical_answers = [{"id": ex["id"], "answers": ex["answers"]} for ex in examples]
    return metric.compute(predictions=predicted_answers, references=theoretical_answers)

In [30]:
# Вычисление метрик

compute_metrics(start_logits, end_logits, eval_set, small_eval_set)

  0%|          | 0/100 [00:00<?, ?it/s]

{'exact_match': 61.0, 'f1': 78.85454822954823}

In [31]:
# Модель

model_checkpoint = 'AndrewChar/model-QA-5-epoch-RU'

model = TFAutoModelForQuestionAnswering.from_pretrained(model_checkpoint)

Some layers from the model checkpoint at AndrewChar/model-QA-5-epoch-RU were not used when initializing TFDistilBertForQuestionAnswering: ['dropout_19']
- This IS expected if you are initializing TFDistilBertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForQuestionAnswering were not initialized from the model checkpoint at AndrewChar/model-QA-5-epoch-RU and are newly initialized: ['dropout_39']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [32]:
# Импорт дата коллатора

from transformers import DefaultDataCollator

data_collator = DefaultDataCollator(return_tensors="tf")

In [33]:
# Создание датасетов

tf_train_dataset = train_dataset.to_tf_dataset(
    columns=[
        "input_ids",
        "start_positions",
        "end_positions",
        "attention_mask",
        "token_type_ids",
    ],
    collate_fn=data_collator,
    shuffle=True,
    batch_size=16,
)
tf_eval_dataset = validation_dataset.to_tf_dataset(
    columns=["input_ids", "attention_mask", "token_type_ids"],
    collate_fn=data_collator,
    shuffle=False,
    batch_size=16,
)

In [34]:
# Гиперпараметры

from transformers import create_optimizer
# from transformers.keras_callbacks import PushToHubCallback
import tensorflow as tf

# The number of training steps is the number of samples in the dataset, divided by the batch size then multiplied
# by the total number of epochs. Note that the tf_train_dataset here is a batched tf.data.Dataset,
# not the original Hugging Face Dataset, so its len() is already num_samples // batch_size.
num_train_epochs = 3
num_train_steps = len(tf_train_dataset) * num_train_epochs
optimizer, schedule = create_optimizer(
    init_lr=2e-5,
    num_warmup_steps=0,
    num_train_steps=num_train_steps,
    weight_decay_rate=0.01,
)
model.compile(optimizer=optimizer)

# Train in mixed-precision float16
tf.keras.mixed_precision.set_global_policy("mixed_float16")

No loss specified in compile() - the model's internal loss computation will be used as the loss. Don't panic - this is a common way to train TensorFlow models in Transformers! To disable this behaviour please pass a loss argument, or explicitly pass `loss=None` if you do not want your model to compute a loss.


In [35]:
# Тренировка модели

model.fit(tf_train_dataset, epochs=num_train_epochs)

Epoch 1/3
2987/2987 [==============================] - 2217s 738ms/step - loss: 1.3393
Epoch 2/3
2987/2987 [==============================] - 2207s 739ms/step - loss: 1.0663
Epoch 3/3
2987/2987 [==============================] - 2204s 738ms/step - loss: 0.8491


In [36]:
# Оценка модели

predictions = model.predict(tf_eval_dataset)

In [48]:
compute_metrics(
    predictions["start_logits"],
    predictions["end_logits"],
    validation_dataset,
    raw_qa_dataset["validation"])

  0%|          | 0/5036 [00:00<?, ?it/s]

{'exact_match': 52.74027005559968, 'f1': 73.88609058054298}

In [51]:
# Проверка работы модели

from transformers import pipeline


In [54]:
question_answerer = pipeline("question-answering", model="mrm8488/bert-multi-cased-finetuned-xquadv1", 
    tokenizer="mrm8488/bert-multi-cased-finetuned-xquadv1")

In [86]:
context = """
Посадил дед репку. Выросла репка большая-пребольшая.

Пошел дед репку рвать: тянет-потянет, вытянуть не может!

Позвал дед бабку:
бабка за дедку,
дедка за репку —
тянут-потянут, вытянуть не могут!

Позвала бабка внучку:
внучка за бабку,
бабка за дедку,
дедка за репку —
тянут-потянут, вытянуть не могут!

Позвала внучка Жучку:
Жучка за внучку,
внучка за бабку,
бабка за дедку,
дедка за репку —
тянут-потянут, вытянуть не могут!

Позвала Жучка кошку:
кошка за Жучку,
Жучка за внучку,
внучка за бабку,
бабка за дедку,
дедка за репку —
тянут-потянут, вытянуть не могут!

Позвала кошка мышку:
мышка за кошку,
кошка за Жучку,
Жучка за внучку,
внучка за бабку,
бабка за дедку,
дедка за репку —
тянут-потянут, — вытянули репку!
"""
question = "Какая репка выросла?"
question_answerer(question=question, context=context)

{'score': 0.42817458510398865,
 'start': 28,
 'end': 52,
 'answer': 'репка большая-пребольшая'}